In [2]:
from shiny import App, render, ui, reactive
import pandas as pd
# import faicons

#* From here to dtype_dict is a dataload, copied in explore.ipynb, app.py, myfunctions.py
# Note: I would have create a data_prep function, but here would have been 20 returned values
excel_file = pd.ExcelFile("stock.xlsx")

# The Excel Sheets loaded in
df_inv = pd.read_excel(excel_file, sheet_name="inventory")
df_check_init = pd.read_excel(excel_file, sheet_name="checkouts")
person_dict = pd.read_excel(excel_file, sheet_name="person_dict")
dept_dict = pd.read_excel(excel_file, sheet_name="dept_dict")
df_copies = pd.read_excel(excel_file, sheet_name="copies_dict")
# Grab details specific to the school:
df_school = pd.read_excel(excel_file, sheet_name="school")
school_name = df_school["School Name"].iloc[0]
background_link = df_school["background_link"].iloc[0]
buffer_amount = df_school["buffer_amount"].iloc[0]
copy_person = df_school["users_name"].iloc[0]

person_list = person_dict["full_name"].unique().tolist()

item_dict = df_inv.set_index("item_id")[["item_name", "price"]].to_dict(orient="index")
inverted_dict = df_inv.set_index("item_name")["item_id"].to_dict()
item_list = df_inv["item_name"].unique().tolist()

# Info Needed for the Copies Tab
add_on_df = df_copies[df_copies["type"] == "add-on"]
add_on_dict = add_on_df.set_index("item_id")["classification"].to_dict()
single_df = df_copies[df_copies["type"] == "single"]
single_dict = single_df.set_index("item_id")["classification"].to_dict()
copies_price = df_copies[["item_id", "type", "price_per_sheet", "classification"]]

# Creating the Dict to tie accounts to People
nested_dict = {}
for _, row in dept_dict.iterrows():
    dept = row["department"]
    account = row["account"]
    number = row["number"]

    if dept not in nested_dict:
        nested_dict[dept] = {}  # Initialize dictionary for the department

    nested_dict[dept][account] = number

acct_options = [] 
acct_options_copies = []
  
dtype_dict = { "item_name": "object", "quantity": "int64", "cost": "float64","memo": "object", "date": "object", "item_id": "int64", "full_name": "object",}
  

In [3]:
from datetime import datetime
import glob
import os
df_check_2 = df_check_init.copy()


def parse_date(date_str):
    try:
        date_str = str(date_str)
        # Try parsing with the format "MM/DD/YY"
        return datetime.strptime(date_str, "%m/%d/%y")
    except ValueError:
        try:
            # Try parsing with the format "MM/DD/YY HH:MM AM/PM"
            return datetime.strptime(date_str, "%m/%d/%y %I:%M %p")
        except ValueError:
            # If neither format works, return NaT
            return pd.NaT


try:
    start = '2024-10-01'
    end = '2024-11-07'
    # Apply the custom parsing function to the 'date' column
    df_check_2["date"] = df_check_2["date"].apply(parse_date)

    # Define the start and end date for filtering
    start_date = datetime.strptime(start, "%Y-%m-%d")
    end_date = datetime.strptime(f"{end} 23:59:59", "%Y-%m-%d %H:%M:%S")
    print(f"start: {start_date}, end: {end_date}")
    print(f"sent_df shape: {df_check_2.shape}")

    # Filter the DataFrame for rows where 'date' is within the specified range
    filtered_df = df_check_2[
        (df_check_2["date"] >= start_date) & (df_check_2["date"] <= end_date)
    ]
    print(f"filterd_df shape: {filtered_df.shape}")

    report_df = filtered_df.groupby("acct", as_index=False)["cost"].sum()
    report_df["cost"] = report_df["cost"].round(2)
    print(f"report_df shape: {report_df.shape}")

    # Add in Account Names
    report_df = report_df.merge(
        dept_dict[["account", "number"]],
        left_on="acct",
        right_on="number",
        how="left",
    )

    # Optionally rename the new column if needed
    report_df.rename(columns={"account": "Account Name"}, inplace=True)
    report_df = report_df.drop(columns=["number"])

    # Format the start and end dates for the filename
    start_str = datetime.strptime(start, "%Y-%m-%d").strftime("%m-%d")
    end_str = datetime.strptime(end, "%Y-%m-%d").strftime("%m-%d")
    filename = f"{start_str}_to_{end_str}_financial_report.xlsx"

    # Delete any existing files that end with "report.xlsx"
    for file in glob.glob("*report.xlsx"):
        os.remove(file)
    # Save report_df to an Excel file with the formatted filename
    report_df.to_excel(filename, index=False)
    print("Done")
except Exception as e:
    print(f"Errored {(e)}")


start: 2024-10-01 00:00:00, end: 2024-11-07 23:59:59
sent_df shape: (6, 8)
filterd_df shape: (6, 8)
report_df shape: (5, 2)
Done


In [2]:
host = "copyroom-checkouts.crmas4440wll.us-west-2.rds.amazonaws.com"
db_user = "admin"
db_password = "Meaning402$$"
database = "checkouts"
port = 3306


import pymysql

db = pymysql.connect(
    host="copyroom-checkouts.crmas4440wll.us-west-2.rds.amazonaws.com",
    user=db_user,
    password=db_password,
    database=database,
    port = port

)